In [1]:
import numpy as np

In [2]:
def gaussian(x, A, u, s):
    return A * np.exp(-((x - u)/s)**2/2)/(np.sqrt(2*np.pi)*s)

In [3]:
x = np.arange(0, 1, 0.001)

In [4]:
def wrap_gaussian(x, A, u, s):
    in_dis = np.abs(x-u)
    x_dis = np.minimum(in_dis, 1-in_dis)
    return gaussian(x_dis, A, 0, s)

In [5]:
data_num = 5000
n_max = 7
A_range = [0.00, 5.00]
u_range = [0.00, 1.00]
s_range = [0.001, 0.01]

In [6]:
def range_sampling(value_range, size):
    return np.random.random(size = size) * (value_range[1] - value_range[0]) + value_range[0]

In [7]:
As = range_sampling(A_range, (data_num, 1, n_max))
us = range_sampling(u_range, (data_num, 1, n_max))
ss = range_sampling(s_range, (data_num, 1, n_max))

In [8]:
data = np.sum(wrap_gaussian(x.reshape(1, -1, 1), As, us, ss), axis = -1)

In [12]:
import torch.nn as nn

In [ ]:
def PConv1d(in_channels, out_channels, padding):
    return nn.Sequential(
                            nn.CircularPad1d(padding),
                            nn.Conv1d(in_channels, out_channels, 2*padding+1)
                        )

def PMaxPool2d(padding):
    return nn.Sequential(
                            nn.CircularPad1d(padding),
                            nn.MaxPool1d()
                        )

In [ ]:
class PAE(nn.Module):
    def __init__(self):
        super(PAE, self).__init__()
        self.encoder = nn.Sequential(
                                        nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
                                        nn.ReLU(),
                                        nn.MaxPool2d(kernel_size=2, stride=2),
                                        nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=1),
                                        nn.ReLU(),
                                        nn.MaxPool2d(kernel_size=2, stride=2)
                                    )
        self.decoder = nn.Sequential(
                                        nn.ConvTranspose2d(8, 16, 
                                                        kernel_size=3, 
                                                        stride=2, 
                                                        padding=1, 
                                                        output_padding=1),
                                        nn.ReLU(),
                                        nn.ConvTranspose2d(16, 3, 
                                                        kernel_size=3, 
                                                        stride=2, 
                                                        padding=1, 
                                                        output_padding=1),
                                        nn.Sigmoid()
                                    )
         
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x